In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import re, os, csv, pathlib, ast
import pandas as pd
from statistics import mean, variance

accent_short_forms = {"hindi":"HIN", "korean":"KOR", "vietnamese":"VTN", "arabic":"ARB", "chinese":"CHN", "spanish":"ESP"}
accent_map = {"ABA":"arabic","SKA":"arabic","YBAA":"arabic","ZHAA":"arabic",
              "BWC":"chinese","LXC":"chinese","NCC":"chinese","TXHC":"chinese",
              "ASI":"hindi","RRBI":"hindi","SVBI":"hindi","TNI":"hindi",
              "HJK":"korean","HKK":"korean","YDCK":"korean","YKWK":"korean",
              "EBVS":"spanish","ERMS":"spanish","MBMPS":"spanish","NJS":"spanish",
              "HQTV":"vietnamese","PNV":"vietnamese","THV":"vietnamese","TLV":"vietnamese"
              }
raw_string="""|ABA|M|Arabic|1129|150|\n|SKA|F|Arabic|974|150|\n|YBAA|M|Arabic|1130|149|\n|ZHAA|F|Arabic|1132|150|\n|BWC|M|Chinese|1130|150|\n|LXC|F|Chinese|1131|150|\n|NCC|F|Chinese|1131|150|\n|TXHC|M|Chinese|1132|150|\n|ASI|M|Hindi|1131|150|\n|RRBI|M|Hindi|1130|150|\n|SVBI|F|Hindi|1132|150|\n|TNI|F|Hindi|1131|150|\n|HJK|F|Korean|1131|150|\n|HKK|M|Korean|1131|150|\n|YDCK|F|Korean|1131|150|\n|YKWK|M|Korean|1131|150|\n|EBVS|M|Spanish|1007|150|\n|ERMS|M|Spanish|1132|150|\n|MBMPS|F|Spanish|1132|150|\n|NJS|F|Spanish|1131|150|\n|HQTV|M|Vietnamese|1132|150|\n|PNV|F|Vietnamese|1132|150|\n|THV|F|Vietnamese|1132|150|\n|TLV|M|Vietnamese|1132|150|"""
raw_strings=raw_string.split('\n')
gender_map={}
for lne in raw_strings:
    attrs=lne.split('|')
    gender_map[attrs[1]]=attrs[2]

composed_accent_map = {k: accent_short_forms.get(v) for k, v in accent_map.items()}

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def group_speakers(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    return str(ret)

def group_accents(s):
    # print(s)
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    # print(ret)
    accent_counts={}
    for accent in ret:
        cnt=0
        for speaker in ret[accent]:
            cnt+=ret[accent][speaker]
        accent_counts[accent]=cnt
    # print(accent_counts)
    return str(accent_counts)

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(group_accents, re.findall('Counter\\((.+?)\\)', s)))+list(map(group_speakers, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    return "-n:"+str(float(eta[4:]))

def clean_cond(fun):
    if 'conditional' not in fun:
        return fun
    _,f,acc=fun.split('_')
    acc=accent_short_forms[acc]
    f=f.split('MI')[0]
    return "{}_cond_against_{}".format(f,acc)

In [3]:
budget = 100
# target = 50
target = 10
features = '39'
# budget, target, features = 100, 50, 'TRILL'
csv_name = "report_{}_{}_{}.csv".format(budget, target, features)

In [4]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/


cols = ['speaker', 'ground', 'function', 'similarity', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-var', 'accents_run1', 'accents_run2', 'accents_run3', 'speakers_run1', 'speakers_run2', 'speakers_run3']
df = pd.DataFrame(columns = cols)

speakers = [f.name for f in os.scandir('./') if f.is_dir() and f.name != '.ipynb_checkpoints' and f.name != 'reserved_TSS_output' and f.name in accent_map]

# within submodular
for speaker in speakers:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        print("no results for speaker {}".format(speaker))
        continue
    if 'within' not in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        print("no all results for {}".format(speaker))
        continue
    if not(pathlib.Path('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)).is_dir()):
        print("no results for budget {} and target {}".format(budget, target))
        continue
    for function in get_dirs('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)):
        if function == "equal_random" or function == speaker or function == "random" or function == "other_random":
            continue
#         for eta in get_dirs('./{}/manifests/TSS_output/all/budget_{}/target_{}/{}/'.format(speaker, budget, target, function)):
        for similarity in get_dirs('./{}/manifests/TSS_output/within/budget_{}/target_{}/{}/'.format(speaker, budget, target, function)):
            stats_file_path = './{}/manifests/TSS_output/within/budget_{}/target_{}/{}/{}/{}/stats.txt'.format(speaker, budget, target, function, similarity, features)  
#                 try:
#                     stats_file = open(stats_file_path, 'r')
#                 except:
#                     continue                                                                                          
            stats_file = open(stats_file_path, 'r')
            lines = stats_file.readlines()
#             df_selections = get_selection_counts(lines[5])
#             total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
#             sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
#             sample_total = mean(total_selections)
#             duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
#             duration_total = mean(total_durations)
#             df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
#             df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
            try:
                wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                df_wer_mean = round(mean(wers), 2)
                df_wer_var = round(variance(wers), 3)
            except:
                print(stats_file_path)
                continue
            speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
            df = df.append(dict(zip(cols, [speaker_new, "within", clean_cond(function), similarity, ["NA"], ["NA"]]+
                                           wers+[df_wer_mean, round(df_wer_var**0.5, 3)] + ["NA"])), 
                           ignore_index=True)
            stats_file.close()

#normal
for speaker in speakers:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        print("no results for speaker {}".format(speaker))
        continue
    if 'all' not in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        print("no all results for {}".format(speaker))
        continue
    if not(pathlib.Path('./{}/manifests/TSS_output/all/budget_{}/target_{}/'.format(speaker, budget, target)).is_dir()):
        print("no results for budget {} and target {}".format(budget, target))
        continue
    for function in get_dirs('./{}/manifests/TSS_output/all/budget_{}/target_{}/'.format(speaker, budget, target)):
        if function == "random":
            continue
        for eta in get_dirs('./{}/manifests/TSS_output/all/budget_{}/target_{}/{}/'.format(speaker, budget, target, function)):
            for similarity in get_dirs('./{}/manifests/TSS_output/all/budget_{}/target_{}/{}/{}/'.format(speaker, budget, target, function, eta)):
                stats_file_path = './{}/manifests/TSS_output/all/budget_{}/target_{}/{}/{}/{}/{}/stats.txt'.format(speaker, budget, target, function, 
                eta, similarity, features)  
#                 try:
#                     stats_file = open(stats_file_path, 'r')
#                 except:
#                     continue                                                                                          
                stats_file = open(stats_file_path, 'r')
                lines = stats_file.readlines()
                df_selections = get_selection_counts(lines[5])
                total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
                sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
                sample_total = mean(total_selections)
                duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
                duration_total = mean(total_durations)
                df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
                df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
                try:
                    wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                    df_wer_mean = round(mean(wers), 2)
                    df_wer_var = round(variance(wers), 3)
                except:
                    print(stats_file_path)
                    continue
                speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
                df = df.append(dict(zip(cols, [speaker_new, "all", clean_cond(function)+get_eta(function, eta), similarity, df_duration, df_samples]+
                                               wers+[df_wer_mean, round(df_wer_var**0.5, 3)] + df_selections)), 
                               ignore_index=True)
                stats_file.close()
                
                
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/random/run_1/output
for speaker in speakers:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        print("no results for speaker {}".format(speaker))
        continue
    if 'all' not in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        print("no all results for {}".format(speaker))
        continue
    if not(pathlib.Path('./{}/manifests/TSS_output/all/budget_{}/target_{}/'.format(speaker, budget, target)).is_dir()):
        print("no results for budget {} and target {}".format(budget, target))
        continue
    if "random" in get_dirs('./{}/manifests/TSS_output/all/budget_{}/target_{}/'.format(speaker, budget, target)):
        stats_file_path = './{}/manifests/TSS_output/all/budget_{}/target_{}/{}/stats.txt'.format(speaker, budget, target, "random")
#         try:
#             stats_file = open(stats_file_path, 'r')
#         except:
#             continue
        stats_file = open(stats_file_path, 'r')
        lines = stats_file.readlines()
        df_selections = get_selection_counts(lines[5])
        total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
        sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
        sample_total = mean(total_selections)
        duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
        duration_total = mean(total_durations)
        df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
        df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
#               print(wers)
        try:
            wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
            df_wer_mean = round(mean(wers), 2)
            df_wer_var = round(variance(wers), 3)
        except:
            print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
            continue
            wers = [0,0,0]
            df_wer_mean = 0
            df_wer_var = 999
        speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
        df = df.append(dict(zip(cols, [speaker_new, "all", "random", "NA", df_duration, df_samples]+wers
                                        +[df_wer_mean, round(df_wer_var**0.5, 3)] + df_selections)), ignore_index=True)
        stats_file.close()

./YKWK/manifests/TSS_output/all/budget_100/target_10/FL1MI/eta_1.0/euclidean/39/stats.txt
./YKWK/manifests/TSS_output/all/budget_100/target_10/GCMI/eta_1.0/euclidean/39/stats.txt
./YDCK/manifests/TSS_output/all/budget_100/target_10/FL1MI/eta_1.0/euclidean/39/stats.txt
./TNI/manifests/TSS_output/all/budget_100/target_10/FL1MI/eta_1.0/euclidean/39/stats.txt
./YBAA/manifests/TSS_output/all/budget_100/target_10/FL1MI/eta_1.0/euclidean/39/stats.txt
./YBAA/manifests/TSS_output/all/budget_100/target_10/GCMI/eta_1.0/euclidean/39/stats.txt
./YBAA/manifests/TSS_output/all/budget_100/target_10/LogDMI/eta_1.0/euclidean/39/stats.txt
./TXHC/manifests/TSS_output/all/budget_100/target_10/FL2MI/eta_1.0/euclidean/39/stats.txt
./TXHC/manifests/TSS_output/all/budget_100/target_10/FL1MI/eta_1.0/euclidean/39/stats.txt
./TXHC/manifests/TSS_output/all/budget_100/target_10/GCMI/eta_1.0/euclidean/39/stats.txt
./TXHC/manifests/TSS_output/all/budget_100/target_10/LogDMI/eta_1.0/euclidean/39/stats.txt
./RRBI/manif

In [5]:
df = df.sort_values(by=['speaker', 'similarity', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3
0,ABA[ARB/M],all,random,NA,14.61/358.25,3.68/100.00,23.76,24.42,23.72,23.97,0.394,"{'KOR': 22, 'ESP': 15, 'VTN': 19, 'ARB': 20, '...","{'KOR': 20, 'CHN': 18, 'VTN': 15, 'ARB': 15, '...","{'KOR': 24, 'ESP': 16, 'ARB': 18, 'CHN': 18, '...","{'KOR': {'YDCK': 7, 'HKK': 7, 'YKWK': 4, 'HJK'...","{'KOR': {'HJK': 9, 'HKK': 5, 'YKWK': 3, 'YDCK'...","{'KOR': {'HJK': 9, 'YDCK': 5, 'HKK': 5, 'YKWK'..."
1,ABA[ARB/M],all,FL1MI-n:1.0,euclidean,267.08/357.67,60.00/83.00,22.33,22.26,22.59,22.39,0.173,"{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ..."
2,ABA[ARB/M],all,FL2MI-n:1.0,euclidean,357.17/357.17,70.00/70.00,21.45,21.20,21.02,21.22,0.217,{'ARB': 70},{'ARB': 70},{'ARB': 70},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}}
3,ABA[ARB/M],all,GCMI-n:1.0,euclidean,359.83/359.83,70.00/70.00,21.71,21.53,21.78,21.67,0.130,{'ARB': 70},{'ARB': 70},{'ARB': 70},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}}
4,ABA[ARB/M],all,LogDMI-n:1.0,euclidean,345.11/358.04,76.00/80.00,21.71,21.89,21.64,21.75,0.130,"{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'...","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'...","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,ZHAA[ARB/F],all,random,NA,20.47/358.25,6.36/100.00,17.99,18.73,18.32,18.35,0.370,"{'KOR': 22, 'ESP': 15, 'VTN': 19, 'ARB': 20, '...","{'KOR': 20, 'CHN': 18, 'VTN': 15, 'ARB': 15, '...","{'KOR': 24, 'ESP': 16, 'ARB': 18, 'CHN': 18, '...","{'KOR': {'YDCK': 7, 'HKK': 7, 'YKWK': 4, 'HJK'...","{'KOR': {'HJK': 9, 'HKK': 5, 'YKWK': 3, 'YDCK'...","{'KOR': {'HJK': 9, 'YDCK': 5, 'HKK': 5, 'YKWK'..."
119,ZHAA[ARB/F],all,FL1MI-n:1.0,euclidean,204.59/358.50,56.00/96.00,18.40,18.40,18.40,18.40,0.000,"{'ARB': 56, 'VTN': 30, 'KOR': 7, 'CHN': 2, 'ES...","{'ARB': 56, 'VTN': 30, 'KOR': 7, 'CHN': 2, 'ES...","{'ARB': 56, 'VTN': 30, 'KOR': 7, 'CHN': 2, 'ES...","{'ARB': {'ZHAA': 54, 'SKA': 2}, 'VTN': {'PNV':...","{'ARB': {'ZHAA': 54, 'SKA': 2}, 'VTN': {'PNV':...","{'ARB': {'ZHAA': 54, 'SKA': 2}, 'VTN': {'PNV':..."
120,ZHAA[ARB/F],all,FL2MI-n:1.0,euclidean,357.67/357.67,95.00/95.00,17.99,17.88,17.84,17.90,0.077,{'ARB': 95},{'ARB': 95},{'ARB': 95},{'ARB': {'ZHAA': 95}},{'ARB': {'ZHAA': 95}},{'ARB': {'ZHAA': 95}}
121,ZHAA[ARB/F],all,GCMI-n:1.0,euclidean,357.43/357.43,94.00/94.00,17.84,17.77,17.73,17.78,0.055,{'ARB': 94},{'ARB': 94},{'ARB': 94},{'ARB': {'ZHAA': 94}},{'ARB': {'ZHAA': 94}},{'ARB': {'ZHAA': 94}}


In [6]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19

df.to_csv(csv_name, index=False)

In [15]:
# speakers = [f.name for f in os.scandir('../accent/') if f.is_dir() and f.name != '.ipynb_checkpoints' and f.name != 'reserved_TSS_output']
# print(speakers)
# for speaker in speakers:
#     for budget in get_dirs('../accent/{}/manifests/TSS_output/all/'.format(speaker)):
#         print(budget)
#         for target in get_dirs('../accent/{}/manifests/TSS_output/all/{}/'.format(speaker, budget)):
#             print(target)